In [1]:
import itertools
import os
import sys
from pathlib import Path

import cv2
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import skimage.io

from collections import defaultdict
from tqdm.auto import tqdm
from joblib import Parallel, delayed
import re
import h5py
import napari
import seaborn as sns

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
p_dir = (Path().cwd().parents[0]).absolute()

module_path = str(p_dir / "src")

if module_path not in sys.path:
    sys.path.append(module_path)

In [4]:
data_dir = (Path().cwd().parents[0] / 'data').absolute()
data_raw = r'Y:\coskun-lab\Shuangyi\ERK, YAP project_2022\PLA\HCC827 cell culture'


In [5]:
from collections import defaultdict

data_dir = (Path().cwd().parents[0] / 'data').absolute()

# Get image dataframe 
df_imgs_path = data_dir / 'OCT Cell Culture' / '3D_Whole' / 'metadata' / 'imgs_sti.csv'
df_imgs = pd.read_csv(df_imgs_path)


# Read per cell data

In [6]:
# Read PPI dfs
PPI_save_path =  data_dir / 'OCT Cell Culture' / '3D_Whole' / 'PPI'

dfs = []
for path in os.listdir(PPI_save_path):
    if 'csv' in path:
        df = pd.read_csv(PPI_save_path / path)
        dfs.append(df)

df = pd.concat(dfs)

# Group by location (Cell vs Nuclei)
df_subset = df[(df.Cell > 0) & (df.Nuclei == 0)]
g = df_subset .groupby(['Condition', 'FOV', 'PPI', 'Cell']).size()
df_cell = pd.DataFrame({'Count Cyto': g}).reset_index()
df_cell.columns = ['Condition', 'FOV', 'PPI', 'Id', 'Count_cell']

df_subset = df[(df.Cell > 0) & (df.Nuclei > 0)]
g = df_subset .groupby(['Condition', 'FOV', 'PPI', 'Cell']).size()
df_nuclei = pd.DataFrame({'Count Nuclei': g}).reset_index()
df_nuclei.columns = ['Condition', 'FOV', 'PPI', 'Id', 'Count_nuclei']

# Concat 
df_all = df_cell.merge(df_nuclei, how='left', on=['Condition', 'FOV', 'PPI', 'Id']).fillna(0)
df_all['Count_nuclei'] = df_all['Count_nuclei'].astype(int)
df_all['Count_cyto'] = df_all['Count_cell'] - df_all['Count_nuclei'] # Cyto count

# Ranme columns
df_all.columns = ['Condition', 'FOV', 'PPI', 'Id', 'Cell', 'Nuclei',
       'Cyto']

# Plot descrition
display(df_all.describe())
display(df_all.groupby(['Condition', 'FOV', 'Id']).sum().describe())

,Id,Cell,Nuclei,Cyto
count,9995.000000,9995.000000,9995.000000,9995.000000
mean,855.548874,14.137969,9.449325,4.688644
std,660.471497,25.335193,10.732661,19.949734
min,21.000000,1.000000,0.000000,-70.000000
25%,358.000000,3.000000,3.000000,-2.000000
50%,649.000000,7.000000,7.000000,1.000000
75%,1187.000000,15.000000,12.000000,6.000000
max,2479.000000,1276.000000,136.000000,1177.000000


,Cell,Nuclei,Cyto
count,2305.000000,2305.000000,2305.00000
mean,61.305423,40.974403,20.33102
std,63.535222,29.606032,49.28650
min,1.000000,0.000000,-86.00000
25%,24.000000,22.000000,-3.00000
50%,46.000000,35.000000,8.00000
75%,80.000000,54.000000,32.00000
max,1415.000000,331.000000,1261.00000


In [7]:
# Filter out by maximum number of counts per cell
min_count = 20
max_count = 70

df_all = df_all.groupby(['Condition', 'FOV', 'Id']).filter(lambda x: x['Cell'].sum() > min_count)
df_all = df_all.groupby(['Condition', 'FOV', 'Id']).filter(lambda x: (x['Cell'] < max_count).all())
# Plot descrition
display(df_all.describe())
display(df_all.groupby(['Condition', 'FOV', 'Id']).sum().describe())

,Id,Cell,Nuclei,Cyto
count,7030.000000,7030.000000,7030.000000,7030.000000
mean,859.473969,12.147511,8.974111,3.173400
std,663.936294,13.078617,8.730724,9.678717
min,21.000000,1.000000,0.000000,-46.000000
25%,339.250000,4.000000,3.000000,-2.000000
50%,656.000000,8.000000,7.000000,1.000000
75%,1212.000000,15.000000,12.000000,6.000000
max,2475.000000,69.000000,86.000000,64.000000


,Cell,Nuclei,Cyto
count,1535.000000,1535.000000,1535.000000
mean,55.633225,41.099674,14.533550
std,27.393821,20.969024,25.183965
min,21.000000,0.000000,-86.000000
25%,34.000000,27.000000,-2.000000
50%,50.000000,38.000000,10.000000
75%,73.000000,52.000000,28.000000
max,219.000000,142.000000,175.000000


# ML model

In [8]:
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn import preprocessing, metrics

from sklearn.model_selection import train_test_split, KFold
from sklearn.utils.class_weight import compute_class_weight
import wandb

In [9]:
condition = 'ML'
project_name = f'PLA_042628_3D_{condition}'

In [10]:
# Get dataframe seperate into count per cell 
df_count = df_all.pivot(index=['Condition', 'FOV', 'Id'], columns=['PPI']).fillna(0)
df_count_cell = df_count.iloc[:, df_count.columns.get_level_values(0)=='Cell']

# Get dataframe seperate into count per nuclei and cyto
df_count_cyto_nuclei = df_count.iloc[:, df_count.columns.get_level_values(0)!='Cell']

# Get Condition into numerical label
le = preprocessing.LabelEncoder()
y = le.fit_transform(df_count.index.get_level_values(0))

In [11]:
df_count

Cell                                         \
PPI                CylinE & CDK2 Mcl-1 & BAK P-ERK & c-MYC TEAD1 & YAP1   
Condition  FOV Id                                                         
HCC827Ctrl FW1 48           22.0         2.0          37.0         10.0   
               52            8.0         0.0          29.0          7.0   
               70           11.0         8.0          26.0         29.0   
               79            9.0         0.0          25.0          7.0   
               80            7.0         0.0           9.0          5.0   
...                          ...         ...           ...          ...   
HCC827Osim FW2 834           3.0         0.0          10.0          8.0   
               837           3.0         0.0          22.0         10.0   
               839           2.0         0.0          11.0          7.0   
               848           1.0         0.0          21.0          6.0   
               857           3.0         0.0          17.0          2.0   

                                       Nuclei                            \
PPI                p-AKT & mTOR CylinE & CDK2 Mcl-1 & BAK P-ERK & c-MYC   
Condition  FOV Id                                                         
HCC827Ctrl FW1 48           6.0          11.0         1.0          13.0   
               52          11.0           4.0         0.0          12.0   
               70           8.0          12.0         3.0           5.0   
               79           1.0          12.0         0.0          17.0   
               80           7.0           3.0         0.0          11.0   
...                         ...           ...         ...           ...   
HCC827Osim FW2 834          0.0           0.0         0.0           0.0   
               837          3.0           3.0         0.0          21.0   
               839          2.0           0.0         0.0          10.0   
               848          5.0           5.0         0.0          17.0   
               857          1.0           0.0         0.0           0.0   

                                                      Cyto              \
PPI                TEAD1 & YAP1 p-AKT & mTOR CylinE & CDK2 Mcl-1 & BAK   
Condition  FOV Id                                                        
HCC827Ctrl FW1 48           8.0          5.0          11.0         1.0   
               52           6.0          2.0           4.0         0.0   
               70          16.0          4.0          -1.0         5.0   
               79           9.0          0.0          -3.0         0.0   
               80           7.0          3.0           4.0         0.0   
...                         ...          ...           ...         ...   
HCC827Osim FW2 834          0.0          0.0           3.0         0.0   
               837         13.0          3.0           0.0         0.0   
               839          6.0          6.0           2.0         0.0   
               848          4.0          3.0          -4.0         0.0   
               857          0.0          0.0           3.0         0.0   

                                                            
PPI                P-ERK & c-MYC TEAD1 & YAP1 p-AKT & mTOR  
Condition  FOV Id                                           
HCC827Ctrl FW1 48           24.0          2.0          1.0  
               52           17.0          1.0          9.0  
               70           21.0         13.0          4.0  
               79            8.0         -2.0          1.0  
               80           -2.0         -2.0          4.0  
...                          ...          ...          ...  
HCC827Osim FW2 834          10.0          8.0          0.0  
               837           1.0         -3.0          0.0  
               839           1.0          1.0         -4.0  
               848           4.0          2.0          2.0  
               857          17.0          2.0          1.0  

[1535 rows x 15 columns]

In [12]:
df_count.reset_index().groupby('Condition').size()

Condition
HCC827Ctrl    886
HCC827Osim    649
dtype: int64

In [19]:
models = {
    'Adaboost': AdaBoostClassifier(),
    'DecisionTree': DecisionTreeClassifier(),
    'GradientBoosting' : GradientBoostingClassifier(),
    'NaiveBayes': GaussianNB(),
    'RandomForest': RandomForestClassifier(), 
    'SVM': SVC(probability =True), 
}

In [24]:
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=0)
kfold = KFold(n_splits = 5, shuffle = True, random_state = 0)

for k, (train_index, test_index) in enumerate(kfold.split(X)):
    # Split the dataset
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    print(y_train.sum(), len(y_train)-y_train.sum(), y_test.sum(), len(y_test)-y_test.sum())

516 712 133 174
520 708 129 178
517 711 132 175
518 710 131 176
525 703 124 183


In [20]:
# Run model on cell count
X = df_count_cell
features = ['_'.join(col) for col in X.columns.values]
X = X.values
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=0)
kfold = KFold(n_splits = 5, shuffle = True, random_state = 0)

for model_name, model in models.items():
    for k, (train_index, test_index) in enumerate(kfold.split(X)):
        # Split the dataset
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]
        
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)
        y_probas = model.predict_proba(X_test)

        run = wandb.init(project=project_name, group=model_name+'_cell', name=model_name+f'_cell_{k}')
        wandb.sklearn.plot_classifier(model, 
                                  X_train, X_test, 
                                  y_train, y_test, 
                                  y_pred, y_probas, 
                                  le.classes_, 
                                  is_binary=True, 
                                  model_name=model_name+'_cell', 
                                  feature_names=features)
        wandb.log({'roc': wandb.plots.ROC(y_test, y_probas, le.classes_)})
        wandb.log({'pr': wandb.plots.precision_recall(y_test, y_probas, le.classes_)})

        accuracy = metrics.accuracy_score(y_test, y_pred)
        f1 = metrics.f1_score(y_test, y_pred)
        auc = metrics.roc_auc_score(y_test, y_pred)
        wandb.log({"accuracy": accuracy, 'f1':f1, 'auc': auc})
    run.finish()

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: thoomas. Use `wandb login --relogin` to force relogin


wandb: 
wandb: Plotting Adaboost_cell.
wandb: Logged feature importances.
wandb: Logged confusion matrix.
wandb: Logged summary metrics.
wandb: Logged class proportions.
wandb: Logged calibration curve.
wandb: WARNING wandb.plots.* functions are deprecated and will be removed in a future release. Please use wandb.plot.* instead.
wandb: Logged roc curve.
wandb: Logged precision-recall curve.


accuracy,▁
auc,▁
f1,▁
accuracy,0.71661
auc,0.70925
f1,0.66667


wandb: 
wandb: Plotting Adaboost_cell.
wandb: Logged feature importances.
wandb: Logged confusion matrix.
wandb: Logged summary metrics.
wandb: Logged class proportions.
wandb: Logged calibration curve.
wandb: Logged roc curve.
wandb: Logged precision-recall curve.


accuracy,▁
auc,▁
f1,▁
accuracy,0.6873
auc,0.67272
f1,0.60976


...

accuracy,▁
auc,▁
f1,▁
accuracy,0.72638
auc,0.70416
f1,0.63158


wandb: 
wandb: Plotting SVM_cell.
wandb: WARNING could not find any of attributes feature_importances_, feature_log_prob_, coef_ on classifier. Cannot plot feature importances.
wandb: Logged feature importances.
wandb: Logged confusion matrix.
wandb: Logged summary metrics.
wandb: Logged class proportions.
wandb: Logged calibration curve.
wandb: Logged roc curve.
wandb: Logged precision-recall curve.


accuracy,▁
auc,▁
f1,▁
accuracy,0.71336
auc,0.69047
f1,0.61404


wandb: 
wandb: Plotting SVM_cell.
wandb: WARNING could not find any of attributes feature_importances_, feature_log_prob_, coef_ on classifier. Cannot plot feature importances.
wandb: Logged feature importances.
wandb: Logged confusion matrix.
wandb: Logged summary metrics.
wandb: Logged class proportions.
wandb: Logged calibration curve.
wandb: Logged roc curve.
wandb: Logged precision-recall curve.


accuracy,▁
auc,▁
f1,▁
accuracy,0.7785
auc,0.75311
f1,0.69369


In [21]:
# Run model on cell count
X = df_count_cyto_nuclei
features = ['_'.join(col) for col in X.columns.values]
X = X.values
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=0)
kfold = KFold(n_splits = 5, shuffle = True, random_state = 0)

for model_name, model in models.items():
    for k, (train_index, test_index) in enumerate(kfold.split(X)):
        # Split the dataset
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]
        
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)
        y_probas = model.predict_proba(X_test)

        run = wandb.init(project=project_name, group=model_name+'_cyto_nuclei', name=model_name+f'_cyto_nuclei_{k}')
        wandb.sklearn.plot_classifier(model, 
                                  X_train, X_test, 
                                  y_train, y_test, 
                                  y_pred, y_probas, 
                                  le.classes_, 
                                  is_binary=True, 
                                  model_name=model_name+'_cyto_nuclei', 
                                  feature_names=features)
        wandb.log({'roc': wandb.plots.ROC(y_test, y_probas, le.classes_)})
        wandb.log({'pr': wandb.plots.precision_recall(y_test, y_probas, le.classes_)})

        accuracy = metrics.accuracy_score(y_test, y_pred)
        f1 = metrics.f1_score(y_test, y_pred)
        auc = metrics.roc_auc_score(y_test, y_pred)
        wandb.log({"accuracy": accuracy, 'f1':f1, 'auc': auc})
    run.finish()


wandb: 
wandb: Plotting Adaboost_cyto_nuclei.
wandb: Logged feature importances.
wandb: Logged confusion matrix.
wandb: Logged summary metrics.
wandb: Logged class proportions.
wandb: Logged calibration curve.
wandb: Logged roc curve.
wandb: Logged precision-recall curve.


accuracy,▁
auc,▁
f1,▁
accuracy,0.76221
auc,0.75923
f1,0.72862


wandb: 
wandb: Plotting Adaboost_cyto_nuclei.
wandb: Logged feature importances.
wandb: Logged confusion matrix.
wandb: Logged summary metrics.
wandb: Logged class proportions.
wandb: Logged calibration curve.
wandb: Logged roc curve.
wandb: Logged precision-recall curve.


accuracy,▁
auc,▁
f1,▁
accuracy,0.72638
auc,0.7107
f1,0.65289


wandb: 
wandb: Plotting Adaboost_cyto_nuclei.
wandb: Logged feature importances.
wandb: Logged confusion matrix.
wandb: Logged summary metrics.
wandb: Logged class proportions.
wandb: Logged calibration curve.
wandb: Logged roc curve.
wandb: Logged precision-recall curve.


...

accuracy,▁
auc,▁
f1,▁
accuracy,0.74919
auc,0.73626
f1,0.68826


wandb: 
wandb: Plotting SVM_cyto_nuclei.
wandb: WARNING could not find any of attributes feature_importances_, feature_log_prob_, coef_ on classifier. Cannot plot feature importances.
wandb: Logged feature importances.
wandb: Logged confusion matrix.
wandb: Logged summary metrics.
wandb: Logged class proportions.
wandb: Logged calibration curve.
wandb: Logged roc curve.
wandb: Logged precision-recall curve.


accuracy,▁
auc,▁
f1,▁
accuracy,0.76221
auc,0.7448
f1,0.69198


wandb: 
wandb: Plotting SVM_cyto_nuclei.
wandb: WARNING could not find any of attributes feature_importances_, feature_log_prob_, coef_ on classifier. Cannot plot feature importances.
wandb: Logged feature importances.
wandb: Logged confusion matrix.
wandb: Logged summary metrics.
wandb: Logged class proportions.
wandb: Logged calibration curve.
wandb: Logged roc curve.
wandb: Logged precision-recall curve.


accuracy,▁
auc,▁
f1,▁
accuracy,0.82085
auc,0.80553
f1,0.76596
